In [0]:
from pyspark.sql.functions import to_date
from pyspark.sql.functions import first
from pyspark.sql.functions import col, round

In [0]:
df_all = spark.read.parquet("dbfs:/databricks-results/bronze/*/*/*")

In [0]:
moedas = ['USD', 'EUR', 'GBP']

df_filter = df_all.filter(df_all.moeda.isin(moedas))

In [0]:
df_filter = df_filter.withColumn("data", to_date("data"))

In [0]:
result_tax = df_filter.groupBy("data").pivot("moeda").agg(first("taxa")).orderBy("data", ascending=False)

In [0]:
df_convert_brl = result_tax.select('*')

In [0]:
for moeda in moedas:
    df_convert_brl = df_convert_brl.withColumn(moeda, round(1/col(moeda), 4))

In [0]:
df_convert_brl = df_convert_brl.coalesce(1)
result_tax = result_tax.coalesce(1)

In [0]:
# Salvando arquivo em csv para melhor visualização

result_tax.write.mode("overwrite").format("csv").option("header", "true").save("dbfs:/databricks-results/prata/taxas_conversao")

df_convert_brl.write.mode("overwrite").format("csv").option("header", "true").save("dbfs:/databricks-results/prata/valores_reais")